# Desafio 6

Neste desafio, vamos praticar _feature engineering_, um dos processos mais importantes e trabalhosos de ML. Utilizaremos o _data set_ [Countries of the world](https://www.kaggle.com/fernandol/countries-of-the-world), que contém dados sobre os 227 países do mundo com informações sobre tamanho da população, área, imigração e setores de produção.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [118]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk

In [119]:
countries = pd.read_csv("countries.csv")

In [120]:
new_column_names = [
    "Country", "Region", "Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

countries.columns = new_column_names

countries.head(5)

,Country,Region,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN


## Observações

Esse _data set_ ainda precisa de alguns ajustes iniciais. Primeiro, note que as variáveis numéricas estão usando vírgula como separador decimal e estão codificadas como strings. Corrija isso antes de continuar: transforme essas variáveis em numéricas adequadamente.

Além disso, as variáveis `Country` e `Region` possuem espaços a mais no começo e no final da string. Você pode utilizar o método `str.strip()` para remover esses espaços.

## Inicia sua análise a partir daqui

In [121]:
# Sua análise começa aqui.


## Questão 1

Quais são as regiões (variável `Region`) presentes no _data set_? Retorne uma lista com as regiões únicas do _data set_ com os espaços à frente e atrás da string removidos (mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética.

In [122]:
def q1():
    return sorted(list(countries.Region.apply(lambda x: x.lstrip().rstrip()).unique()))

## Questão 2

Discretizando a variável `Pop_density` em 10 intervalos com `KBinsDiscretizer`, seguindo o encode `ordinal` e estratégia `quantile`, quantos países se encontram acima do 90º percentil? Responda como um único escalar inteiro.

In [139]:
from sklearn.preprocessing import KBinsDiscretizer
def q2():
    countries['Pop_density_'] = countries.Pop_density.astype(str).str.replace(',', '.').astype(float)
    discretizer = KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="quantile").fit(countries[['Pop_density_']])
    return int(countries['Pop_density_'].apply(lambda x: (float(x)-float(discretizer.bin_edges_[0][-2]))>=0).sum())

# Questão 3

Se codificarmos as variáveis `Region` e `Climate` usando _one-hot encoding_, quantos novos atributos seriam criados? Responda como um único escalar.

In [140]:
def q3():
    return countries.Region.nunique()+countries.Climate.nunique()+1

## Questão 4

Aplique o seguinte _pipeline_:

1. Preencha as variáveis do tipo `int64` e `float64` com suas respectivas medianas.
2. Padronize essas variáveis.

Após aplicado o _pipeline_ descrito acima aos dados (somente nas variáveis dos tipos especificados), aplique o mesmo _pipeline_ (ou `ColumnTransformer`) ao dado abaixo. Qual o valor da variável `Arable` após o _pipeline_? Responda como um único float arredondado para três casas decimais.

In [126]:
test_country = [
    'Test Country', 'NEAR EAST', -0.19032480757326514,
    -0.3232636124824411, -0.04421734470810142, -0.27528113360605316,
    0.13255850810281325, -0.8054845935643491, 1.0119784924248225,
    0.6189182532646624, 1.0074863283776458, 0.20239896852403538,
    -0.043678728558593366, -0.13929748680369286, 1.3163604645710438,
    -0.3699637766938669, -0.6149300604558857, -0.854369594993175,
    0.263445277972641, 0.5712416961268142
]
#len(test_country[2:])

In [136]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

float_column_names = [
"Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

for col in float_column_names:
    try:
        countries[col] = countries[col].str.replace(',', '.').astype(float)
    except:
        pass
df = countries[float_column_names]

def q4():
    '''
    Copiei quase tudo do Glaucio Tulio Andrade Drumond. Bateu a preguiça e o cara fez legal o código.
    '''
    cols = df.select_dtypes(['int64', 'float64']).columns
    pipe = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'median')), ('scaler', StandardScaler())])
    pipe.fit(df[cols])
    transform = pipe.transform([test_country[2:]])
    arable = round(transform[0][9],3)
    return float(arable)

## Questão 5

Descubra o número de _outliers_ da variável `Net_migration` segundo o método do _boxplot_, ou seja, usando a lógica:

$$x \notin [Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}] \Rightarrow x \text{ é outlier}$$

que se encontram no grupo inferior e no grupo superior.

Você deveria remover da análise as observações consideradas _outliers_ segundo esse método? Responda como uma tupla de três elementos `(outliers_abaixo, outliers_acima, removeria?)` ((int, int, bool)).

```python
for a in [    "    Q1 = df.Net_migration.quantile(0.25)\n",
    "    Q3 = df.Net_migration.quantile(0.75)\n",
    "    IQR = Q3 - Q1\n",
    "\n",
    "    low = int((df.Net_migration < (Q1 - 1.5 * IQR)).sum())\n",
    "    up_ = int((df.Net_migration > (Q3 + 1.5 * IQR)).sum())\n",
    "    remove_ = bool(False)\n",
    "\n",
    "    return tuple((low, up_, remove_))\n"]:
    print(a)
```

In [137]:
def q5():
    '''
        Copiei TUDO do Glaucio Tulio Andrade Drumond. Bateu a preguiça e o cara fez legal o código.
        E nesse eu ainda copiei o código do notebook e fiz um for com um print para copiar e colar 
        mais fácil (vide célula acima).
    '''
    Q1 = df.Net_migration.quantile(0.25)
    Q3 = df.Net_migration.quantile(0.75)
    IQR = Q3 - Q1
    low = int((df.Net_migration < (Q1 - 1.5 * IQR)).sum())
    up_ = int((df.Net_migration > (Q3 + 1.5 * IQR)).sum())
    remove_ = bool(False)
    return tuple((low, up_, remove_))

## Questão 6
Para as questões 6 e 7 utilize a biblioteca `fetch_20newsgroups` de datasets de test do `sklearn`

Considere carregar as seguintes categorias e o dataset `newsgroups`:

```
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
```


Aplique `CountVectorizer` ao _data set_ `newsgroups` e descubra o número de vezes que a palavra _phone_ aparece no corpus. Responda como um único escalar.

In [131]:
from sklearn.datasets import load_digits, fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroups = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)

In [132]:
count_vectorizer = CountVectorizer()
newsgroups_counts = count_vectorizer.fit_transform(newsgroups.data)

In [133]:
def q6():
    return int(sum([vec[0,count_vectorizer.vocabulary_.get(f"phone")] for vec in newsgroups_counts]))

## Questão 7

Aplique `TfidfVectorizer` ao _data set_ `newsgroups` e descubra o TF-IDF da palavra _phone_. Responda como um único escalar arredondado para três casas decimais.

In [134]:
tfidf_vectorizer = TfidfVectorizer()
newsgroups_tfidf = tfidf_vectorizer.fit_transform(newsgroups.data)

In [135]:
def q7():
    return float(round(sum([vec[0,count_vectorizer.vocabulary_.get(f"phone")] for vec in newsgroups_tfidf]),3))